In [1]:
%load_ext autoreload
%autoreload 2

# Goal: Generate V&V targets for the facility choice model

The targets are output in a `.csv` file at the end of the notebook.

## Requirements:

If you want to run this notebook or any of the code in the
`birth_facility.py` or `solution_finding.py` modules, you will need the right environment.
You can create it with these steps (on 64-bit Linux):

- `conda create --name facility_choice --file environment_lock_conda.txt`
- `conda activate facility_choice`
- `pip install -r environment_lock_pip.txt`

If you're not on 64-bit Linux, replacing the first command with `conda create --name facility_choice python=3.10`
should approximately recreate the environment.

If you want to update all the packages in the environment, create it using `conda env create --name facility_choice --file environment.yaml` (no need for any `pip` commands).
Then, run the following commands inside the environment you created to update the lock files:

- `conda list --explicit > environment_lock_conda.txt`
- `pip freeze | grep -v 'file:///' | grep -v '\-e' > environment_lock_pip.txt`

You will also need to be on the IHME cluster to run this notebook, as the causal model
nanosim loads LBWSG data from an Artifact for the MNCNH Portfolio model.

In [2]:
import pandas as pd
import numpy as np
import birth_facility as bf

!date
!whoami
!uname -a
!pwd
!python --version
!conda info --envs | grep '\*'
!conda list | grep -e pandas -e numpy -e scipy -e statsmodels -e matplotlib -e gbd

Thu Nov 13 09:03:56 PST 2025
zmbc
Linux long-slurm-sarchive-p0056 5.4.0-216-generic #236-Ubuntu SMP Fri Apr 11 19:53:21 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/homes/zmbc/src/vivarium_research_mncnh_portfolio/facility_choice
Python 3.10.19
facility_choice       *  /mnt/share/homes/zmbc/mambaforge/envs/facility_choice
gbd                       4.37.1                   pypi_0    pypi
gbd-mapping               5.0.0                    pypi_0    pypi
matplotlib-inline         0.1.7                    pypi_0    pypi
numpy                     1.26.4                   pypi_0    pypi
pandas                    1.5.3                    pypi_0    pypi
pandas-stubs              2.2.3.250308             pypi_0    pypi
scipy                     1.15.3                   pypi_0    pypi
statsmodels               0.14.5                   pypi_0    pypi


In [3]:
# Import a few specific things for convenience
from birth_facility import (
    # Classes for causal model nanosim
    BirthFacilityChoiceData, BirthFacilityModelWithUltrasoundAndSimpleGAError,
    # String indicating fake draw column that's the mean of the real draws
    MEAN_DRAW,
    # Constants for column names
    ANC, LBWSG_CAT, FACILITY, TERM_STATUS, BELIEVED_TERM_STATUS, ULTRASOUND,
)

# Part 1: Run a nanosim for each country

The 5 input parameters are the outputs previously found by the causal
model optimization. The 5 inputs are 3 correlations and 2 causal
probabilities. These same parameters should be used in Vivarium.

# Load results of optimization for reference

In [4]:
# These will be the inputs to the causal model nanosims below
optimization_results = pd.read_csv('facility_choice_optimization_results.csv').set_index('parameter_name')
optimization_results

,Ethiopia,Nigeria,Pakistan
parameter_name,,,
"corr(anc, lbwsg_category)",0.200000,0.200000,0.200000
"corr(anc, facility)",0.795975,0.455476,0.361761
"corr(lbwsg_category, facility)",0.200000,0.200000,0.200000
prob_home_given_believed_preterm,0.204066,0.295749,0.129769
prob_home_given_believed_term,0.619051,0.630588,0.295613



# Load data for the 3 countries

In [5]:
draw = MEAN_DRAW

data_p = BirthFacilityChoiceData('pakistan', draw=draw)
data_e = BirthFacilityChoiceData('ethiopia', draw=draw)
data_n = BirthFacilityChoiceData('nigeria', draw=draw)

# Define propensity correlations, create models for all 3 countries, and sample propensities



In [6]:
# Choose a large population size to reduce stochastic variation for
# validation targets
pop_size = 1_000_000

# These maps specify the three propensity correlations for each country
correlation_map_e = {
    (ANC, FACILITY): optimization_results.loc['corr(anc, facility)', 'Ethiopia'],
    (ANC, LBWSG_CAT): optimization_results.loc['corr(anc, lbwsg_category)', 'Ethiopia'],
    (FACILITY, LBWSG_CAT): optimization_results.loc['corr(lbwsg_category, facility)', 'Ethiopia'],
}
correlation_map_n = {
    (ANC, FACILITY): optimization_results.loc['corr(anc, facility)', 'Nigeria'],
    (ANC, LBWSG_CAT): optimization_results.loc['corr(anc, lbwsg_category)', 'Nigeria'],
    (FACILITY, LBWSG_CAT): optimization_results.loc['corr(lbwsg_category, facility)', 'Nigeria'],
}
correlation_map_p = {
    (ANC, FACILITY): optimization_results.loc['corr(anc, facility)', 'Pakistan'],
    (ANC, LBWSG_CAT): optimization_results.loc['corr(anc, lbwsg_category)', 'Pakistan'],
    (FACILITY, LBWSG_CAT): optimization_results.loc['corr(lbwsg_category, facility)', 'Pakistan'],
}

# Create a causal model nanosim for each country
model_e = BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data=data_e,
    population_size=pop_size,
    num_facility_types=2,
    correlation_map=correlation_map_e,
    seed=303916847607905435497113962748943192257,
)
model_n = BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data=data_n,
    population_size=pop_size,
    num_facility_types=2,
    correlation_map=correlation_map_n,
    seed=291258168795962949552887306725641490503,
)
model_p = BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data=data_p,
    population_size=pop_size,
    num_facility_types=2,
    correlation_map=correlation_map_p,
    seed=106517498220144615105209854865147031104,
)

# Display correlation matrix for Ethiopia:
# The entries should match the correlations specified in correlation_map
# above
model_e.correlation_matrix

,sex,anc,ultrasound,lbwsg_category,gestational_age,ga_error,facility
sex,1.0,0.000000,0.0,0.0,0.0,0.0,0.000000
anc,0.0,1.000000,0.0,0.2,0.0,0.0,0.795975
ultrasound,0.0,0.000000,1.0,0.0,0.0,0.0,0.000000
lbwsg_category,0.0,0.200000,0.0,1.0,0.0,0.0,0.200000
gestational_age,0.0,0.000000,0.0,0.0,1.0,0.0,0.000000
ga_error,0.0,0.000000,0.0,0.0,0.0,1.0,0.000000
facility,0.0,0.795975,0.0,0.2,0.0,0.0,1.000000


## Sample propensities and display result for Ethiopia

Then verify that the empirical correlations match the inputs.

In [7]:
model_e.sample_propensities()
model_n.sample_propensities()
model_p.sample_propensities()

model_e.propensity

,sex,anc,ultrasound,lbwsg_category,gestational_age,ga_error,facility
simulant_id,,,,,,,
0,0.821092,0.999763,0.537079,0.898939,0.282300,0.995695,0.997433
1,0.730453,0.747400,0.028765,0.828203,0.887033,0.882391,0.425245
2,0.063495,0.881214,0.454569,0.206587,0.358215,0.298074,0.966601
3,0.274937,0.568044,0.789880,0.934536,0.124789,0.004025,0.754390
4,0.950339,0.017551,0.672279,0.338996,0.625502,0.115858,0.064503
...,...,...,...,...,...,...,...
999995,0.841760,0.581793,0.656507,0.894732,0.657879,0.159309,0.400187
999996,0.448645,0.947309,0.922859,0.346518,0.307209,0.884578,0.870190
999997,0.834919,0.036408,0.262872,0.138283,0.600444,0.648388,0.197781


In [8]:
# Check empirical propensity correlations - these should approximately
# match the correlation matrix above.
# I also added a method to do this (empirical_propensity_correlations),
# but I'm writing the code explicitly here to show that it's using the
# sampled propensities.
pd.DataFrame(
    np.corrcoef(model_e.propensity, rowvar=False),
    index=model_e.propensity.columns,
    columns=model_e.propensity.columns,
).round(2)


,sex,anc,ultrasound,lbwsg_category,gestational_age,ga_error,facility
sex,1.0,0.00,-0.0,-0.00,0.0,-0.0,0.00
anc,0.0,1.00,0.0,0.19,0.0,0.0,0.78
ultrasound,-0.0,0.00,1.0,0.00,0.0,-0.0,0.00
lbwsg_category,-0.0,0.19,0.0,1.00,-0.0,0.0,0.19
gestational_age,0.0,0.00,0.0,-0.00,1.0,-0.0,0.00
ga_error,-0.0,0.00,-0.0,0.00,-0.0,1.0,0.00
facility,0.0,0.78,0.0,0.19,0.0,0.0,1.00


# Define causal facility choice probabilities, and initialize populations

In [9]:
# These are P(home|do(believed preterm)) and P(home|do(believed term))
model_e.set_facility_choice_probabilities(optimization_results.loc['prob_home_given_believed_preterm', 'Ethiopia'], optimization_results.loc['prob_home_given_believed_term', 'Ethiopia'])
model_n.set_facility_choice_probabilities(optimization_results.loc['prob_home_given_believed_preterm', 'Nigeria'], optimization_results.loc['prob_home_given_believed_term', 'Nigeria'])
model_p.set_facility_choice_probabilities(optimization_results.loc['prob_home_given_believed_preterm', 'Pakistan'], optimization_results.loc['prob_home_given_believed_term', 'Pakistan'])

model_e.assign_population_variables()
model_n.assign_population_variables()
model_p.assign_population_variables()

# Display population for Ethiopia
model_e.population

,sex,anc,ultrasound,lbwsg_category,gestational_age,term_status,ga_error,estimated_gestational_age,believed_term_status,facility
simulant_id,,,,,,,,,,
0,Female,anc1,standard_ultrasound,cat54,38.564600,term,11.259329,49.823929,believed_term,in_facility
1,Female,anc1,no_ultrasound,cat54,39.774066,term,11.870263,51.644329,believed_term,at_home
2,Male,anc1,standard_ultrasound,cat117,38.716429,term,-2.271206,36.445223,believed_preterm,in_facility
3,Male,anc1,standard_ultrasound,cat54,38.249577,term,-11.356996,26.892581,believed_preterm,in_facility
4,Female,anc0,no_ultrasound,cat44,39.251003,term,-11.959495,27.291508,believed_preterm,at_home
...,...,...,...,...,...,...,...,...,...,...
999995,Female,anc1,standard_ultrasound,cat54,39.315758,term,-4.274146,35.041612,believed_preterm,in_facility
999996,Male,anc1,standard_ultrasound,cat42,37.307209,term,5.135081,42.442290,believed_term,in_facility
999997,Female,anc0,no_ultrasound,cat116,39.200889,term,3.809707,43.010596,believed_term,at_home


# Part 2: Compute validation targets

Some validation targets are inputs to the causal model optimization,
and some are computed from the simulated population after running the
nanosims above.

# Display the optimization targets for the 3 countries

These are the conditional probabilities that the optimization tries to
match. When we run the facility choice model in Vivarium using the five
inputs specified above, the simulated population should approximately
reproduce these values.

In [10]:
models = [model_e, model_n, model_p]
locations = [model.data.location for model in models]

optimization_targets = pd.concat(
    [model.targets for model in models],
    axis=1, keys=locations, names=['location']
)
optimization_targets

location                    Ethiopia   Nigeria  Pakistan
probability_of given                                    
in_facility    anc0         0.183369  0.309069  0.556322
               anc1         0.664786  0.584168  0.795407
preterm        at_home      0.163579  0.172663  0.194529
               in_facility  0.163486  0.153000  0.155164

In [11]:
# Reshape for concatenation with other validation targets:
# Unstack the variables in the "probability_of" column to get a table of
# conditional probabilities in the same format as output by my
# `prob_y_given_x` function (see below), then use my stacking function
# to get a single-level index with appropriate labels.
reshaped_optimization_targets = bf.stack_conditional_probability_table(
    optimization_targets.unstack('probability_of'))
reshaped_optimization_targets

location,Ethiopia,Nigeria,Pakistan
probability_of,,,
in_facility_given_anc0,0.183369,0.309069,0.556322
in_facility_given_anc1,0.664786,0.584168,0.795407
preterm_given_at_home,0.163579,0.172663,0.194529
preterm_given_in_facility,0.163486,0.153000,0.155164


### For comparison, display the empirical target values achieved by the nanosim populations:

They look pretty close, but not perfect. This should give some idea of
how close Vivarium should get to the target values.

In [12]:
pd.concat(
    [model.calculate_targets() for model in models],
    axis=1, keys=locations, names=['location']
)

location                    Ethiopia   Nigeria  Pakistan
probability_of given                                    
in_facility    anc0         0.187114  0.312543  0.560029
               anc1         0.665468  0.582752  0.795093
preterm        at_home      0.164750  0.174078  0.195641
               in_facility  0.163001  0.152140  0.155431

# Display input probabilities for the 3 countries

These are the exact probabilities that were used as input somewhere in
the model, directly or indirectly.

In [13]:
input_probabilities = pd.concat(
    [model.data.input_probabilities for model in models],
    axis=1, keys=locations, names=['location']
)
# These are already in the right shape for concatenation
input_probabilities

location,Ethiopia,Nigeria,Pakistan
subpopulation,,,
Male,0.515312,0.512450,0.519811
Female,0.484688,0.487550,0.480189
anc0,0.219890,0.257001,0.075919
anc1,0.780110,0.742999,0.924081
preterm,0.163527,0.162567,0.163933
term,0.836473,0.837433,0.836067
at_home,0.441073,0.486533,0.222744
in_facility,0.558927,0.513467,0.777256
BEmONC,0.089922,0.002271,0.264677


### For comparison, display the empirical population proportions in the nanosims:

Note that I'm running the 2-facility-type model, which doesn't include
BEmONC and CEmONC.

In [14]:
pd.concat(
    [model.get_population_proportions() for model in models],
    axis=1, keys=locations, names=['location']
)

location,Ethiopia,Nigeria,Pakistan
subpopulation,,,
Male,0.515319,0.513159,0.520580
Female,0.484681,0.486841,0.479420
anc0,0.220529,0.256528,0.075755
anc1,0.779471,0.743472,0.924245
preterm,0.163771,0.162814,0.164386
term,0.836229,0.837186,0.835614
at_home,0.440023,0.486564,0.222714
in_facility,0.559977,0.513436,0.777286
BEmONC,NaN,NaN,NaN


# Display observed probabilities of IFD status given believed preterm status

These probabilities, $\Pr(\text{IFD status} \mid \text{believed preterm status})$, are computed
from the simulated population and should match what is observed in the
Vivarium sim.

These probabilities are different from the corresponding causal
probabilities $\Pr(\text{IFD status} \mid
\operatorname{do}(\text{believed preterm status}))$ that were input
above, because of the nonzero correlations of IFD status with ANC
attendance and LBWSG category.

In [15]:
observed_ifd_probabilities_given_believed_term_status = pd.concat(
    [model.get_observed_facility_choice_probabilities() for model in models],
    axis=1,
    keys=locations,
    names=['location']
)
observed_ifd_probabilities_given_believed_term_status

location              Ethiopia               Nigeria              Pakistan  \
facility               at_home in_facility   at_home in_facility   at_home   
believed_term_status                                                         
believed_preterm      0.223254    0.776746  0.314611    0.685389  0.141032   
believed_term         0.602512    0.397488  0.615716    0.384284  0.282238   

location                          
facility             in_facility  
believed_term_status              
believed_preterm        0.858968  
believed_term           0.717762

In [16]:
# Reshape for concatenation with other validation targets
reshaped_observed_ifd_probabilities_given_believed_term_status = (
    bf.stack_conditional_probability_table(
        observed_ifd_probabilities_given_believed_term_status)
)
reshaped_observed_ifd_probabilities_given_believed_term_status

location,Ethiopia,Nigeria,Pakistan
probability_of,,,
at_home_given_believed_preterm,0.223254,0.314611,0.141032
in_facility_given_believed_preterm,0.776746,0.685389,0.858968
at_home_given_believed_term,0.602512,0.615716,0.282238
in_facility_given_believed_term,0.397488,0.384284,0.717762


### For comparison, display the causal probabilities we input when we initialized the populations:

These are different from the observed values, as expected.

In [17]:
pd.concat(
    [model.prob_facility_given_believed_term_status for model in models],
    axis=1, keys=locations, names=['location']
).pipe(bf.stack_conditional_probability_table)

location,Ethiopia,Nigeria,Pakistan
probability_of,,,
at_home_given_believed_preterm,0.204066,0.295749,0.129769
in_facility_given_believed_preterm,0.795934,0.704251,0.870231
at_home_given_believed_term,0.619051,0.630588,0.295613
in_facility_given_believed_term,0.380949,0.369412,0.704387


# Display confusion matrices of term status vs. believed term status

I'm using the term [confusion
matrix](https://en.wikipedia.org/wiki/Confusion_matrix) somewhat
loosely...

These probabilities, $\Pr(\text{believed preterm status} \mid \text{preterm status})$, are computed
from the simulated population and should match what is observed in the
Vivarium sim.

The values here will vary by location because of differing ultrasound
rates.

These probabilities depend on the [AI Ultrasound
module](https://vivarium-research.readthedocs.io/en/latest/models/concept_models/vivarium_mncnh_portfolio/ai_ultrasound_module/module_document.html#vivarium-mncnh-portfolio-ai-ultrasound-module)
and the LBWSG exposure but don't involve the facility choice probabilities.

In [18]:
term_status_confusion = [
    bf.prob_y_given_x(
        model.population[BELIEVED_TERM_STATUS],
        model.population[TERM_STATUS]
    ) for model in models
]

prob_believed_term_status_given_term_status = pd.concat(
    term_status_confusion, axis=1, keys=locations)
prob_believed_term_status_given_term_status


Ethiopia                        Nigeria  \
believed_term_status believed_preterm believed_term believed_preterm   
term_status                                                            
preterm                      0.657815      0.342185         0.653304   
term                         0.383518      0.616482         0.385291   

                                           Pakistan                
believed_term_status believed_term believed_preterm believed_term  
term_status                                                        
preterm                   0.346696         0.677692      0.322308  
term                      0.614709         0.371145      0.628855

In [19]:
# Reshape for concatenation with other validation targets
reshaped_prob_believed_term_status_given_term_status = (
    bf.stack_conditional_probability_table(
        prob_believed_term_status_given_term_status)
)
reshaped_prob_believed_term_status_given_term_status

,Ethiopia,Nigeria,Pakistan
probability_of,,,
believed_preterm_given_preterm,0.657815,0.653304,0.677692
believed_term_given_preterm,0.342185,0.346696,0.322308
believed_preterm_given_term,0.383518,0.385291,0.371145
believed_term_given_term,0.616482,0.614709,0.628855


# Display P(believed term status | term status, ultrasound)

These probabilities, $\Pr(\text{believed preterm status} \mid
\text{preterm status, ultrasound type})$, are computed
from the simulated population and should match what is observed in the
Vivarium sim.

Should these values be independent of location?? They look close enough
that the differences could be due to stochastic variation. I think maybe
they shouldn't be exactly the same because of differences in the
gestational age distribution.

These probabilities depend on the [AI Ultrasound
module](https://vivarium-research.readthedocs.io/en/latest/models/concept_models/vivarium_mncnh_portfolio/ai_ultrasound_module/module_document.html#vivarium-mncnh-portfolio-ai-ultrasound-module)
and the LBWSG exposure but don't involve the facility choice probabilities.

In [20]:
ultrasound_tables = [
    bf.prob_y_given_x(
        model.population[BELIEVED_TERM_STATUS],
        model.population[[TERM_STATUS, ULTRASOUND]],
    ) for model in models
]

prob_believed_term_status_given_term_status_and_ultrasound = pd.concat(
    ultrasound_tables, axis=1, keys=locations)
prob_believed_term_status_given_term_status_and_ultrasound


Ethiopia                \
believed_term_status            believed_preterm believed_term   
term_status ultrasound                                           
preterm     no_ultrasound               0.616378      0.383622   
            standard_ultrasound         0.716382      0.283618   
term        no_ultrasound               0.427392      0.572608   
            standard_ultrasound         0.336780      0.663220   

                                         Nigeria                \
believed_term_status            believed_preterm believed_term   
term_status ultrasound                                           
preterm     no_ultrasound               0.615546      0.384454   
            standard_ultrasound         0.716558      0.283442   
term        no_ultrasound               0.426289      0.573711   
            standard_ultrasound         0.334963      0.665037   

                                        Pakistan                
believed_term_status            believed_preterm believed_term  
term_status ultrasound                                          
preterm     no_ultrasound               0.618615      0.381385  
            standard_ultrasound         0.719810      0.280190  
term        no_ultrasound               0.426544      0.573456  
            standard_ultrasound         0.337494      0.662506

In [21]:
# Reshape for concatenation with other validation targets
reshaped_prob_believed_term_status_given_term_status_and_ultrasound = (
    bf.stack_conditional_probability_table(
        prob_believed_term_status_given_term_status_and_ultrasound)
)
reshaped_prob_believed_term_status_given_term_status_and_ultrasound


,Ethiopia,Nigeria,Pakistan
probability_of,,,
believed_preterm_given_preterm_and_no_ultrasound,0.616378,0.615546,0.618615
believed_term_given_preterm_and_no_ultrasound,0.383622,0.384454,0.381385
believed_preterm_given_preterm_and_standard_ultrasound,0.716382,0.716558,0.719810
believed_term_given_preterm_and_standard_ultrasound,0.283618,0.283442,0.280190
believed_preterm_given_term_and_no_ultrasound,0.427392,0.426289,0.426544
believed_term_given_term_and_no_ultrasound,0.572608,0.573711,0.573456
believed_preterm_given_term_and_standard_ultrasound,0.336780,0.334963,0.337494
believed_term_given_term_and_standard_ultrasound,0.663220,0.665037,0.662506


In [22]:
# Compute average probabilities for the 3 countries
sum(ultrasound_tables) / 3

believed_term_status             believed_preterm  believed_term
term_status ultrasound                                          
preterm     no_ultrasound                0.616846       0.383154
            standard_ultrasound          0.717583       0.282417
term        no_ultrasound                0.426742       0.573258
            standard_ultrasound          0.336412       0.663588

# Concatenate all the validation targets

In [23]:
tables = [
    reshaped_optimization_targets,
    input_probabilities,
    reshaped_observed_ifd_probabilities_given_believed_term_status,
    reshaped_prob_believed_term_status_given_term_status,
    reshaped_prob_believed_term_status_given_term_status_and_ultrasound,
]

validation_targets = (
    pd.concat(tables)
    .rename_axis(index='probability_of', columns='location')
    .rename(index=lambda s: s.lower())
)
validation_targets

location,Ethiopia,Nigeria,Pakistan
probability_of,,,
in_facility_given_anc0,0.183369,0.309069,0.556322
in_facility_given_anc1,0.664786,0.584168,0.795407
preterm_given_at_home,0.163579,0.172663,0.194529
preterm_given_in_facility,0.163486,0.153000,0.155164
male,0.515312,0.512450,0.519811
female,0.484688,0.487550,0.480189
anc0,0.219890,0.257001,0.075919
anc1,0.780110,0.742999,0.924081
preterm,0.163527,0.162567,0.163933


# Save validation targets to .csv and verify format by reading back in

It looks like the "location" name for the columns gets dropped, but I
guess we don't actually need it...

In [24]:
output_filename = "facility_choice_validation_targets.csv"
validation_targets.to_csv(output_filename)
pd.read_csv(output_filename, index_col=0)

,Ethiopia,Nigeria,Pakistan
probability_of,,,
in_facility_given_anc0,0.183369,0.309069,0.556322
in_facility_given_anc1,0.664786,0.584168,0.795407
preterm_given_at_home,0.163579,0.172663,0.194529
preterm_given_in_facility,0.163486,0.153000,0.155164
male,0.515312,0.512450,0.519811
female,0.484688,0.487550,0.480189
anc0,0.219890,0.257001,0.075919
anc1,0.780110,0.742999,0.924081
preterm,0.163527,0.162567,0.163933
